### grp

## Hands-On Machine Learning with Scikit-Learn & TensorFlow

## CHAPTER 8: Dimensionality Reduction

#### it is recommended to first train original system before considering dimensionality reduction

### High Dimensional Dataset Challenges:
-  "curse of dimensionality":
    -  slower training
    -  difficult to find solutions / visualize patterns
    -  data can be very sparse
    -  less reliable predictions due to data points being far away from one another
    -  high risk of overfitting

### High Dimensional Dataset Problem Solving:
-  increase size of training set
-  DR feature engineering

### DR Benefits:
-  may filter out noise
-  improve training speed and performance
-  useful for data visualization from N dimensions to 2 or 3 dimensions to identify patterns (ex: clusters)

### DR Approaches:
-  Projection:
    -  all training lie close to lower-dimensional subspace of the high-dimensional space
    -  visualizations on page 210
    -  this approach is not recommended for twist and turn datasets like _swiss roll_ example
-  Manifold Learning:
    -  _swiss roll_ example is a 2D manifold (2D shape that can be bent and twisted in an N-dimensional space)
    -  **manifold hypothesis** => most high-dimensional datasets like close to a much lower-dimensional manifold

### DR Algorithms:

https://en.wikipedia.org/wiki/Principal_component_analysis

-  PCA:
    -  identifies the hyperplane that lies closest to the data and projects the data
    -  ***principal component*** => unit vector that defines the ith axis
    -  1st PC - identifies the axis that accounts for the largest amount of variance in the training set
    -  2nd PC - identifies the axis that accounts for the next largest amount of remaining variance
    -  visualizations on page 214
    -  PCA finds more axes based on the number of dimensions on the dataset
    -  SVD (***Singular Value Decomposition***) https://en.wikipedia.org/wiki/Singular_value_decomposition
    -  PCA assumes the dataset is centered around the origin
-  PCA Compression:
    -  after DR training set takes up less space:
        -  example => preserving 95% variance on MNIST dataset:
            -  original 784 features; now 154 features
            -  option to decompress dataset back to 784 dimensions via inverse transformation
            -  ***reconstruction error*** => mean squared distance between compressed and decompressed data
-  Incremental PCA:
    -  regular PCA requires entire training set to fit in memory
    -  incremental method splits the training set into mini-batches
-  Randomized PCA:
    -  stochastic algorithm that finds approximation of 1st principal components

***quote***: "Once you have identified all the principal components, you can reduce the dimensionality of the dataset down to d dimensions by projecting it onto the hyperplane defined by the first d principal components. Selecting this hyperplane ensures that the projection will preserve as much variance as possible. To project the training set onto the hyperplane, you can simply compute the dot product of the training set matrix X by the matrix Wd, defined as the matrix containing the first d principal components (i.e., the matrix composed of the first d columns of VT)" - _hands on ml w sklearn and tf (aurelien geron)_

-  Kernel PCA:
    -  unsupervised learning algorithm (no obvious performance measure to select best kernel and hyperparameter values)
    -  similar to the "kernel trick" via SVM
    -  performs complex nonlinear projections for dimensionality reduction
    -  good at preserving clusters of instances after projection as well as unrolling datasets in twisted manifold
-  LLE:
    -  aka "Locally Linear Embedding"
    -  manifold learning technique
    -  does not rely on projections like kernel PCA
    -  measures how each training instance linearly relates to its closest neighbors (c.n.) then ...:
        -  looks for low-dimensional representation of the training set where local relationships are best preserved
    -  works well with twisted datasets (ex: _swiss roll_)

### Choosing Right Number of Dimensions:
-  idea is to preserve as much variance of the training set data (ex: 95%)
-  recommended to choose the number of dimensions that add up to a sufficiently large portion of the variance (ex: 95%)
-  for visualization reduce the dimensionality down to 2 or 3

### Other DR Techniques:
-  Multidimensional Scaling (MDS) 
-  Isomap
-  t-Distributed Stochastic Neighbor Embedding (t-SNE)
-  Linear Discriminant Analysis (LDA)

## _Exercises_

In [1]:
import sklearn
print(sklearn.__version__)

0.20.0


### svd

In [2]:
import numpy as np 
np.random.seed(4)
m = 60
w1, w2 = 0.1, 0.3
noise = 0.1

# 3d dataset
angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
X = np.empty((m, 3))
X[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
X[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
X[:, 2] = X[:, 0] * w1 + X[:, 1] * w2 + noise * np.random.randn(m)

In [3]:
#extract first 2 principal components of training set
X_centered = X - X.mean(axis=0)
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

### hyperplane projection

In [4]:
# projects the training set onto the plane defined by the first 2 principal components
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

### sklearn pca

In [5]:
# uses SVD decomposition
    # ex: reduce dimensionality of the dataset down to 2 dimensions

from sklearn.decomposition import PCA

pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

In [6]:
X2D[:5]

array([[ 1.26203346,  0.42067648],
       [-0.08001485, -0.35272239],
       [ 1.17545763,  0.36085729],
       [ 0.89305601, -0.30862856],
       [ 0.73016287, -0.25404049]])

In [7]:
# horizontal vector
pca.components_

array([[-0.93636116, -0.29854881, -0.18465208],
       [ 0.34027485, -0.90119108, -0.2684542 ]])

In [8]:
# first principal component
pca.components_.T[:,0]

array([-0.93636116, -0.29854881, -0.18465208])

In [9]:
# represents the proportion of the dataset's variance that lies along the axis of each principal component:
    # 3D dataset ratios:
        # first axis => represents 84.2% of the dataset's variance
        # second axis => represents 14.6% of the dataset's variance
        # third axis => represents 1.2% of the dataset's variance (thus carrying little information)
pca.explained_variance_ratio_

array([0.84248607, 0.14631839])

### preserve variance options

In [10]:
from six.moves import urllib
from sklearn.datasets import fetch_mldata
try:
    mnist = fetch_mldata('MNIST original')
except urllib.error.HTTPError as ex:
    print("Could not download MNIST data from mldata.org, trying alternative...")

    # Alternative method to load MNIST, if mldata.org is down
    from scipy.io import loadmat
    mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
    mnist_path = "./mnist-original.mat"
    response = urllib.request.urlopen(mnist_alternative_url)
    with open(mnist_path, "wb") as f:
        content = response.read()
        f.write(content)
    mnist_raw = loadmat(mnist_path)
    mnist = {
        "data": mnist_raw["data"].T,
        "target": mnist_raw["label"][0],
        "COL_NAMES": ["label", "data"],
        "DESCR": "mldata.org dataset: mnist-original",
    }
    print("Success!")

/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22
  warnings.warn(msg, category=DeprecationWarning)


In [11]:
from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
# option 1
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
print(d) # min number of dimensions required to preserve 95% of the training set's variance

154


In [13]:
# option 2
pca = PCA(n_components=0.95) # indicates ratio of variance to preserve
X_reduced = pca.fit_transform(X_train)
print(pca.n_components_)

154


In [14]:
np.sum(pca.explained_variance_ratio_)

0.9504463030200189

### decompression

In [15]:
pca = PCA(n_components = 154)
X_reduced = pca.fit_transform(X_train)
X_recovered = pca.inverse_transform(X_reduced)

### incremental pca

In [16]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_train, n_batches):
    print(".", end="") # not shown in the book
    inc_pca.partial_fit(X_batch)

X_reduced = inc_pca.transform(X_train)

....................................................................................................

### incremental pca numpy method

In [17]:
# let's create the memmap() structure and copy the MNIST data into it. This would typically be done by a first program
filename = "my_mnist.data"
m, n = X_train.shape

X_mm = np.memmap(filename, dtype='float32', mode='write', shape=(m, n))
X_mm[:] = X_train

In [18]:
del X_mm # now deleting the memmap() object will trigger its Python finalizer, which ensures that the data is saved to disk

In [19]:
# another program would load the data and use it for training
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

IncrementalPCA(batch_size=525, copy=True, n_components=154, whiten=False)

### randomized pca

In [20]:
rnd_pca = PCA(n_components=154, svd_solver="randomized", random_state=42)
X_reduced = rnd_pca.fit_transform(X_train)

### kernel pca

In [21]:
'''
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)
'''

'\nfrom sklearn.decomposition import KernelPCA\n\nrbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.04)\nX_reduced = rbf_pca.fit_transform(X)\n'

### kernel tuning hyperparameters:
1.  reduce dimensionality to 2 dimensions via kPCA
2.  apply logistic regression for classification
3.  use GridSearchCV (cross validation) to find best kernel and gamma value for kPCA
4.  all of these steps help find the best classification accuracy

In [22]:
print(X.shape)
print(X[:10])
print(X.ndim)

(70000, 784)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
2


In [23]:
print(y.shape)
print(y[:10])
print(y.ndim)

(70000,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1


In [24]:
'''
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression(solver="liblinear"))
    ])

param_grid = [{
        "kpca__gamma": np.linspace(0.03, 0.05, 10),
        "kpca__kernel": ["rbf", "sigmoid"]
    }]

grid_search = GridSearchCV(clf, param_grid, cv=3)
grid_search.fit(X, y)
'''

'\nfrom sklearn.decomposition import KernelPCA\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.pipeline import Pipeline\n\nclf = Pipeline([\n        ("kpca", KernelPCA(n_components=2)),\n        ("log_reg", LogisticRegression(solver="liblinear"))\n    ])\n\nparam_grid = [{\n        "kpca__gamma": np.linspace(0.03, 0.05, 10),\n        "kpca__kernel": ["rbf", "sigmoid"]\n    }]\n\ngrid_search = GridSearchCV(clf, param_grid, cv=3)\ngrid_search.fit(X, y)\n'

In [25]:
'''
print(grid_search.best_params_)
'''

'\nprint(grid_search.best_params_)\n'

### reconstruction pre-image

In [26]:
'''
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433,
                    fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)
'''

'\nrbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0433,\n                    fit_inverse_transform=True)\nX_reduced = rbf_pca.fit_transform(X)\nX_preimage = rbf_pca.inverse_transform(X_reduced)\n'

In [27]:
'''
from sklearn.metrics import mean_squared_error

mean_squared_error(X, X_preimage)
'''

'\nfrom sklearn.metrics import mean_squared_error\n\nmean_squared_error(X, X_preimage)\n'

### lle

In [28]:
'''
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)
X_reduced = lle.fit_transform(X)
'''

'\nfrom sklearn.manifold import LocallyLinearEmbedding\n\nlle = LocallyLinearEmbedding(n_components=2, n_neighbors=10, random_state=42)\nX_reduced = lle.fit_transform(X)\n'

### additional exercises:

https://github.com/ageron/handson-ml/blob/master/08_dimensionality_reduction.ipynb

1. What are the main motivations for reducing a dataset’s dimensionality? What are the main drawbacks?
2. What is the curse of dimensionality?
3. Once a dataset’s dimensionality has been reduced, is it possible to reverse the operation? If so, how? If not, why?
4. Can PCA be used to reduce the dimensionality of a highly nonlinear dataset?
5. Suppose you perform PCA on a 1,000-dimensional dataset, setting the explained variance ratio to 95%. How many dimensions will the resulting dataset have?
6. In what cases would you use vanilla PCA, Incremental PCA, Randomized PCA, or Kernel PCA?
7. How can you evaluate the performance of a dimensionality reduction algorithm on your dataset?
8. Does it make any sense to chain two different dimensionality reduction algorithms?
9. Load the MNIST dataset (introduced in Chapter 3) and split it into a training set and a test set (take the first 60,000 instances for training, and the remaining 10,000 for testing). Train a Random Forest classifier on the dataset and time how long it takes, then evaluate the resulting model on the test set. Next, use PCA to reduce the dataset’s dimensionality, with an explained variance ratio of 95%. Train a new Random Forest classifier on the reduced dataset and see how long it takes. Was training much faster? Next evaluate the classifier on the test set: how does it compare to the previous classifier?
10. Use t-SNE to reduce the MNIST dataset down to two dimensions and plot the result using Matplotlib. You can use a scatterplot using 10 different colors to represent each image’s target class. Alternatively, you can write colored digits at the location of each instance, or even plot scaled-down versions of the digit images themselves (if you plot all digits, the visualization will be too cluttered, so you should either draw a random sample or plot an instance only if no other instance has already been plotted at a close distance). You should get a nice visualization with well-separated clusters of digits. Try using other dimensionality reduction algorithms such as PCA, LLE, or MDS and compare the resulting visualizations.

### grp